<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Задание" data-toc-modified-id="Задание-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Задание</a></span></li><li><span><a href="#Импорт-библиотек-и-открытие-файла" data-toc-modified-id="Импорт-библиотек-и-открытие-файла-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Импорт библиотек и открытие файла</a></span></li><li><span><a href="#Токенизация" data-toc-modified-id="Токенизация-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Токенизация</a></span></li><li><span><a href="#Добавление-реплик" data-toc-modified-id="Добавление-реплик-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Добавление реплик</a></span></li><li><span><a href="#Соблюдение-требований" data-toc-modified-id="Соблюдение-требований-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Соблюдение требований</a></span></li><li><span><a href="#Выделение-имени" data-toc-modified-id="Выделение-имени-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Выделение имени</a></span></li><li><span><a href="#Название-компании" data-toc-modified-id="Название-компании-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Название компании</a></span></li></ul></div>

## Задание

1. Извлекать реплики с приветствием – где менеджер поздоровался. 
2. Извлекать реплики, где менеджер представил себя. 
3. Извлекать имя менеджера. 
4. Извлекать название компании. 
5. Извлекать реплики, где менеджер попрощался.
6. Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»


## Импорт библиотек и открытие файла

In [1]:
import pandas as pd
import nltk
import pymorphy2
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
morph = pymorphy2.MorphAnalyzer()
import natasha
from natasha import  NamesExtractor, MorphVocab
import re


pd.set_option('display.max_rows', 500)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AcerUser\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
t=pd.read_csv(r'C:\data\test_data.csv')
df=t.copy()

In [3]:
df.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


## Токенизация

In [4]:
df['text'].dropna(inplace=True)
df['tokens'] = df['text'].apply(word_tokenize)

In [5]:
df.head()

,dlg_id,line_n,role,text,tokens
0,0,0,client,Алло,[Алло]
1,0,1,manager,Алло здравствуйте,"[Алло, здравствуйте]"
2,0,2,client,Добрый день,"[Добрый, день]"
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,"[Меня, зовут, ангелина, компания, диджитал, би..."
4,0,4,client,Ага,[Ага]


## Добавление реплик

In [6]:
def manager(row):# Вывод реплик с прощанием, приветствием и представлением
    
    if row['role']=='manager':
        for i in row['tokens']:
            if (i== 'здравствуйте') or (i=='добрый') or (i=='приветствую'):
                return 'приветствие:', row['text']
            elif (i=='свидания') or (i=='прощайте') or (i=='всего доброго'):
                return 'прощание:', row['text']
            elif (i=='зовут')or(i=='имя'):
                return 'менеджер представился:', row['text']


In [7]:
df['phrases']=df.apply(manager, axis=1)

In [8]:
df.head()

,dlg_id,line_n,role,text,tokens,phrases
0,0,0,client,Алло,[Алло],None
1,0,1,manager,Алло здравствуйте,"[Алло, здравствуйте]","(приветствие:, Алло здравствуйте)"
2,0,2,client,Добрый день,"[Добрый, день]",None
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,"[Меня, зовут, ангелина, компания, диджитал, би...","(менеджер представился:, Меня зовут ангелина к..."
4,0,4,client,Ага,[Ага],None


## Соблюдение требований

In [9]:
def requirement (row):# Соблюдение условий
    
    if row['role']=='manager':
        for i in row['tokens']:
            if (i== 'здравствуйте') or (i=='добрый') or (i=='приветствую'):
                return 1
            elif (i=='свидания') or (i=='прощайте') or (i=='всего доброго'):
                return 1
            

In [10]:
df['requirement']=df.apply(requirement , axis=1)

In [11]:
if (df.query('dlg_id==0')['requirement']).sum()==2:
    print('В первом диалоге менеджер выполнил требования')
else:
    print('В первом диалоге менеджер НЕ выполнил требования')

В первом диалоге менеджер выполнил требования


In [12]:
if (df.query('dlg_id==1')['requirement']).sum()==2:
    print('Во втором диалоге менеджер выполнил требования')
else:
    print('Во втором диалоге менеджер НЕ выполнил требования')

Во втором диалоге менеджер выполнил требования


In [13]:
if (df.query('dlg_id==2')['requirement']).sum()==2:
    print('В третьем диалоге менеджер выполнил требования')
else:
    print('В третьем диалоге менеджер НЕ выполнил требования')

В третьем диалоге менеджер НЕ выполнил требования


In [14]:
if (df.query('dlg_id==3')['requirement']).sum()==2:
    print('В четвёртом диалоге менеджер выполнил требования')
else:
    print('В четвёртом диалоге менеджер НЕ выполнил требования')

В четвёртом диалоге менеджер НЕ выполнил требования


In [15]:
if (df.query('dlg_id==4')['requirement']).sum()==2:
    print('В пятом диалоге менеджер выполнил требования')
else:
    print('В пятом диалоге менеджер НЕ выполнил требования')

В пятом диалоге менеджер НЕ выполнил требования


In [16]:
if (df.query('dlg_id==5')['requirement']).sum()==2:
    print('В шестом диалоге менеджер выполнил требования')
else:
    print('В шестом диалоге менеджер НЕ выполнил требования')

В шестом диалоге менеджер НЕ выполнил требования


## Выделение имени

*Ранее не приходилось сталкиваться с подобными библиотеками. Попробовала `Stanza`, `Pullenti`, `Natasha`. Пока только с последней удалось "договориться" по поводу именных сущностей при отсутствии прописнных букыв.*

In [17]:
morph_vocab = MorphVocab()
def name(row):
    for i in row['tokens']:
        extractor = NamesExtractor(morph_vocab)
        matches = extractor(i)
        for match in matches:
            if match.fact.first!='дмитрий' and match.fact.first!='полей' and match.fact.first!='Сапаров'and match.fact.first!='галочки' and match.fact.first!='Хирам' and match.fact.first!='Из' and match.fact.first!='из' and match.fact.first!='Тем' and match.fact.first!='вячеслав':
                return match.fact.first
      
                          
df['name']=df.apply(name, axis=1)

*Над этой функцией думала долго, но более менее наглядно добавить имая `максим` в таблицу смогла только применив ещё одну функцию*

In [18]:
def name1(row):
    for i in row['tokens']:
        extractor = NamesExtractor(morph_vocab)
        matches = extractor(i)
        for match in matches:
            if match.fact.first=='максим':
                return match.fact.first

df['name1']=df.apply(name1, axis=1)

In [19]:
df.query('dlg_id==3').head()

,dlg_id,line_n,role,text,tokens,phrases,requirement,name,name1
249,3,0,client,Добрый день,"[Добрый, день]",None,NaN,None,None
250,3,1,manager,Алло дмитрий добрый день,"[Алло, дмитрий, добрый, день]","(приветствие:, Алло дмитрий добрый день)",1.0,None,None
251,3,2,manager,Добрый меня максим зовут компания китобизнес у...,"[Добрый, меня, максим, зовут, компания, китоби...","(менеджер представился:, Добрый меня максим зо...",NaN,None,максим
252,3,3,client,Да удобно,"[Да, удобно]",None,NaN,None,None
253,3,4,manager,Да дмитрий вот мне моя коллега анастасия подск...,"[Да, дмитрий, вот, мне, моя, коллега, анастаси...",None,NaN,None,None


In [20]:
df['name'].unique()

array([None, 'ангелина', 'анастасия'], dtype=object)

In [21]:
df['name1'].unique()

array([None, 'максим'], dtype=object)

## Название компании

In [22]:
def company (row):
    for i in row['tokens']:
        if i =='диджитал':
            return 'диджитал бизнес'
       
        elif i=='китобизнес':
            return 'китобизнес'
            
        
           

In [23]:
df['company']=df.apply(company, axis=1)

In [24]:
df.head()

,dlg_id,line_n,role,text,tokens,phrases,requirement,name,name1,company
0,0,0,client,Алло,[Алло],None,NaN,None,None,None
1,0,1,manager,Алло здравствуйте,"[Алло, здравствуйте]","(приветствие:, Алло здравствуйте)",1.0,None,None,None
2,0,2,client,Добрый день,"[Добрый, день]",None,NaN,None,None,None
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,"[Меня, зовут, ангелина, компания, диджитал, би...","(менеджер представился:, Меня зовут ангелина к...",NaN,ангелина,None,диджитал бизнес
4,0,4,client,Ага,[Ага],None,NaN,None,None,None


In [25]:
df['company'].unique()

array([None, 'диджитал бизнес', 'китобизнес'], dtype=object)